In [ ]:
#!/usr/bin/env python3
import os
import sqlite3
import pandas as pd
from glob import glob
from tqdm import tqdm

# --------------------------
# 配置部分
# --------------------------
# 手动指定待扫描的文件夹列表（这些文件夹中存放 -mmsi.csv 文件）
dir_list = [
    "/mnt/nas/fan/ais_ripe_log/2022/01/STA",
    "/mnt/nas/fan/ais_ripe_log/2022/02/STA"
]

# 数据库文件（这里使用一个 SQLite 数据库文件，里面包含两个表）
db_file = "data/mmsi_database.db"

In [6]:
# --------------------------
# 数据库初始化
# --------------------------

conn = sqlite3.connect(db_file)
cursor = conn.cursor()

# 建立存储 MMSI 记录的表，以 MMSI 为主键
cursor.execute('''
CREATE TABLE IF NOT EXISTS mmsi_records (
    MMSI TEXT PRIMARY KEY,
    MsgId TEXT,
    ShipType TEXT,
    ShipName TEXT,
    Callsign TEXT
)
''')

# 建立记录已处理文件的表，文件名作为主键
cursor.execute('''
CREATE TABLE IF NOT EXISTS processed_files (
    filename TEXT PRIMARY KEY,
    error_count INTEGER,
    total_count INTEGER
)
''')
conn.commit()

# 关闭数据库连接
conn.close()

In [ ]:
# --------------------------
# 扫描目录并处理文件
# --------------------------

conn = sqlite3.connect(db_file)
cursor = conn.cursor()

for directory in dir_list:
    # 搜索以“-mmsi.csv”结尾的文件
    pattern = os.path.join(directory, "*-mmsi.csv")
    file_list = glob(pattern)
    
    for file_path in file_list:
        # 检查当前文件是否已经处理过
        cursor.execute("SELECT 1 FROM processed_files WHERE filename = ?", (file_path,))
        if cursor.fetchone():
            print(f"File {file_path} 已经处理过，跳过。")
            continue
        
        print(f"正在处理文件：{file_path}")
        try:
            # 读取 CSV 文件
            df = pd.read_csv(file_path, delimiter=",")
        except Exception as e:
            print(f"读取文件 {file_path} 时出错：{e}")
            continue
        
        total_count = len(df)
        error_count = 0
        
        # 遍历每一行记录，使用 tqdm 显示进度
        for index, row in tqdm(df.iterrows(), total=total_count, desc=f"处理 {os.path.basename(file_path)}"):
            mmsi = str(row['MMSI'])
            # 尝试从数据库中查找该 MMSI 记录
            cursor.execute("SELECT MsgId, ShipType, ShipName, Callsign FROM mmsi_records WHERE MMSI = ?", (mmsi,))
            result = cursor.fetchone()
            
            if result is None:
                # 如果数据库中不存在该 MMSI，则直接插入
                cursor.execute(
                    "INSERT INTO mmsi_records (MMSI, MsgId, ShipType, ShipName, Callsign) VALUES (?,?,?,?,?)",
                    (mmsi, str(row['MsgId']), str(row['ShipType']), str(row['ShipName']), str(row['Callsign']))
                )
            else:
                # 如果已存在，则检查其他字段是否一致
                db_msgid, db_shiptype, db_shipname, db_callsign = result
                new_values = (str(row['MsgId']), str(row['ShipType']), str(row['ShipName']), str(row['Callsign']))
                if (str(db_msgid), str(db_shiptype), str(db_shipname), str(db_callsign)) != new_values:
                    error_count += 1
                    
        # 提交本次文件的记录更新
        conn.commit()
        
        # 将当前文件标记为已处理
        cursor.execute(
            "INSERT INTO processed_files (filename, error_count, total_count) VALUES (?,?,?)",
            (file_path, error_count, total_count)
        )
        conn.commit()
        
        error_ratio = error_count / total_count if total_count > 0 else 0
        print(f"文件 {file_path} 处理完毕：错误 {error_count}/{total_count}，错误率 {error_ratio:.2%}")

# 关闭数据库连接
conn.close()

正在处理文件：/mnt/nas/fan/ais_ripe_log/2022/01/STA/2022-01-15-mmsi.csv


处理 2022-01-15-mmsi.csv: 100%|██████████| 775992/775992 [00:23<00:00, 32982.85it/s]


文件 /mnt/nas/fan/ais_ripe_log/2022/01/STA/2022-01-15-mmsi.csv 处理完毕：错误 156766/775992，错误率 20.20%
正在处理文件：/mnt/nas/fan/ais_ripe_log/2022/01/STA/2022-01-01-mmsi.csv


处理 2022-01-01-mmsi.csv: 100%|██████████| 754908/754908 [00:24<00:00, 31051.96it/s]


文件 /mnt/nas/fan/ais_ripe_log/2022/01/STA/2022-01-01-mmsi.csv 处理完毕：错误 169768/754908，错误率 22.49%
正在处理文件：/mnt/nas/fan/ais_ripe_log/2022/01/STA/2022-01-02-mmsi.csv


...

文件 /mnt/nas/fan/ais_ripe_log/2022/01/STA/2022-01-05-mmsi.csv 处理完毕：错误 184081/787683，错误率 23.37%
正在处理文件：/mnt/nas/fan/ais_ripe_log/2022/01/STA/2022-01-06-mmsi.csv


处理 2022-01-06-mmsi.csv: 100%|██████████| 772118/772118 [00:23<00:00, 32784.44it/s]


文件 /mnt/nas/fan/ais_ripe_log/2022/01/STA/2022-01-06-mmsi.csv 处理完毕：错误 178495/772118，错误率 23.12%
正在处理文件：/mnt/nas/fan/ais_ripe_log/2022/01/STA/2022-01-07-mmsi.csv


/tmp/ipykernel_3211049/2298892858.py:23: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, delimiter=",")
处理 2022-01-07-mmsi.csv:  91%|█████████ | 705944/775830 [00:22<00:02, 31558.89it/s]


KeyboardInterrupt: 

In [8]:
# --------------------------
# 查询数据库内容
# --------------------------

conn = sqlite3.connect(db_file)
cursor = conn.cursor()

# 查询并展示 mmsi_records 表中的所有内容
df_records = pd.read_sql_query("SELECT * FROM mmsi_records", conn)
print("=== mmsi_records 表内容 ===")
print(df_records)

# 查询并展示 processed_files 表中的所有内容
df_processed = pd.read_sql_query("SELECT * FROM processed_files", conn)
print("\n=== processed_files 表内容 ===")
print(df_processed)

# 关闭数据库连接
conn.close()

=== mmsi_records 表内容 ===
             MMSI MsgId ShipType      ShipName Callsign
0       244630581     5       79       AB ALIO   PG3902
1       413459540     5       70  SHI  HUA  66     BQJI
2       229353000     5       80    SEAEXPRESS  9HA3260
3       416459000     5       74   UNI-PROSPER     BKHC
4       413451640     5       60   SHENG HE 16      nan
...           ...   ...      ...           ...      ...
300481  503035630   24B       36           nan   JV100N
300482  338389484   24B       37           nan   WDC965
300483  412443619   24B       30           nan      nan
300484  412449462   24B       30           nan   000000
300485  431016712   24B       30           nan      nan

[300486 rows x 5 columns]

=== processed_files 表内容 ===
                                            filename  error_count  total_count
0  /mnt/nas/fan/ais_ripe_log/2022/01/STA/2022-01-...       156766       775992
1  /mnt/nas/fan/ais_ripe_log/2022/01/STA/2022-01-...       169768       754908
2  /mnt/na